Welcome to my fantasy football player prediction system. In this code i will use machine learning to predict how many points a player should get in an upcoming gameweek. I will use a classfier which will produce high accruacy results based on the data which has been trained by a user. 

Firslty we need to load the neccessary packages.

In [2]:
import os
import csv 
import pandas as pd
import glob 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
directory_path = 'PremierLeagueData/2022-23/understat/'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

dataframes = []

for file in csv_files:
    if 'understat_' not in file:
        df = pd.read_csv(file)
        player_name = os.path.basename(file).split('.')[0].replace('_',' ')
        position = df['position']
        
        h_team = df['h_team']
        a_team = df['a_team']
            
        df['player_name'] = player_name
        df['h_team'] = h_team
        df['a_team'] = a_team
        df['position'] = position
        dataframes.append(df)
        
players = pd.concat(dataframes, ignore_index=True)

players = players.groupby(['id', 'player_name', 'position', 'h_team', 'a_team'], as_index=False).agg('sum')

players.to_csv('players.csv', index=False)

print(players)

          id            player_name position             h_team      a_team  \
0         81       Ashley Young 631      AML  Manchester United   Tottenham   
1         81         Ben Davies 660       DL  Manchester United   Tottenham   
2         81  Christian Eriksen 646      AMC  Manchester United   Tottenham   
3         81          Dele Alli 645      Sub  Manchester United   Tottenham   
4         81          Eric Dier 643      DMC  Manchester United   Tottenham   
...      ...                    ...      ...                ...         ...   
58278  19904      Djed Spence 10764       DR             Rennes   Marseille   
58279  19909      Djed Spence 10764       DR            Auxerre      Rennes   
58280  19909   Siriki Dembélé 10749      Sub            Auxerre      Rennes   
58281  19911        Yan Valery 7280       DC             Angers    Toulouse   
58282  19916     Morgan Sanson 3696       MC          Marseille  Strasbourg   

       goals  shots        xG  time  h_goals  a_goa

I will now drop any player that started off as a sub because this will affect the accuracy score and if a player who normally starts but is rested for one week and is started as sub then this won't give an accurate representation.

Data Preprocessing & Feature Engieneering

Data Preprocessing - This part will involve me cleaning, transforming and selecting the data which will be relevent for me. 

Feature Engineering- Creating any other bits of data to improve my prediction

Firstly i'm going to separate the goalkeeper, these will need different features to predict the points off as the main focus for there points will be around keeping clean sheets. If a goal keeper keeps a clean sheet they acheive 5 points. 

After separeting the goal keepers i'm now going to separate the defenders. I will be introducing goals and assists into this as some defenders reguarly get assists as they are very attacking minded.

Now i'm separating the midfielders, this will included the same features as the defenders because if a midfielder keeps a clean sheet they earn an extra point. 

Finally i'm seperating the forwards which won't include goals condeded as if a forwards team concedes they don't lose any points. The main feature for these will be goals scored and assists becuase that will be where the majority of their points come from. 

Model Selection- Selecting my ML algorithm which is most appropriate and achieves the highest accuracy. 

In [4]:
df = pd.read_csv("players.csv")

#dropping any player who's starting position is sub
df = df[df['position'] != 'Sub']

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals only
y = last_10_games[["goals"]]

In [5]:
# Encode the position column using LabelEncoder
le = LabelEncoder()
last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])

C:\Users\aaron\AppData\Local\Temp\ipykernel_1640\1006643639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])


In [6]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X.values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = regressor.predict(X_test)

C:\Users\aaron\AppData\Local\Temp\ipykernel_1640\376972348.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


In [7]:
# Create an empty list to store predicted values for each player
preds_list = []

for player_id in df['player_name'].unique():
    player_data = df[df['player_name'] == player_id].tail(10)  # select last 10 games for each player
    X = scaler.transform(player_data[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]].values)
    
    y = player_data['goals']
    pred = regressor.predict(X)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2 = r2_score(y, pred)
    # Append the predicted value to the preds_list
    preds_list.append({'player_name': player_id, 'predicted_goals': round(pred[0], 2)})

top_predicted = pd.DataFrame(preds_list).sort_values(by='predicted_goals', ascending=False)
print(top_predicted.head(10))

C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\l

               player_name  predicted_goals
265        Phil Foden 6055             2.86
392         Ivan Toney 998             2.02
459   Julián Álvarez 10846             2.00
17        Riyad Mahrez 750             2.00
334     Taiwo Awoniyi 7814             1.98
208   Dominic Solanke 1679             1.00
317     Eddie Nketiah 6482             1.00
198       Solly March 6049             1.00
430             Tetê 10536             1.00
72   Kelechi Iheanacho 620             1.00


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [8]:
# Create a dataframe from the preds_list
preds = pd.DataFrame(preds_list)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

Mean squared error:  1.295896328293739e-06
Mean absolute error:  6.479481641468689e-05
R-squared value:  0.9999885932495689


In [9]:
#GRID SEARCH

# param_grid = {
#     'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
#     'activation': ['logistic', 'tanh', 'relu'],
#     'solver': ['adam', 'sgd'],
#     'learning_rate': ['constant', 'adaptive'],
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
# }


# # # Perform grid search
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train_scaled, y_train)

# # # Get the best parameters
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

NEW MODEL USING MY OWN MLP CLASSIFIER

In [10]:
df = pd.read_csv("players.csv")

#dropping any player who's starting position is sub
df = df[df['position'] != 'Sub']

# Filter the data to get the last 10 games for each player
last_10_games = df.groupby("player_name").tail(10)

# Select the features for X
X = last_10_games[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]]

# Create the y matrix for goals only
y = last_10_games[["goals"]]

In [11]:
# Encode the position column using LabelEncoder
le = LabelEncoder()
last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])

C:\Users\aaron\AppData\Local\Temp\ipykernel_1640\1006643639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_games["position_encoded"] = le.fit_transform(last_10_games["position"])


In [12]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X.values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the architecture of your MLP
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model on your training data
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Make predictions on the test set
y_pred = model.predict(X_test)

Epoch 1/50
232/232 [==============================] - 2s 2ms/step - loss: 0.0210
Epoch 2/50
232/232 [==============================] - 1s 2ms/step - loss: 0.0030
Epoch 3/50
232/232 [==============================] - 1s 4ms/step - loss: 0.0016
Epoch 4/50
232/232 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 5/50
232/232 [==============================] - 1s 2ms/step - loss: 9.1217e-04
Epoch 6/50
232/232 [==============================] - 0s 2ms/step - loss: 7.2948e-04
Epoch 7/50
232/232 [==============================] - 0s 2ms/step - loss: 6.0713e-04
Epoch 8/50
232/232 [==============================] - 0s 2ms/step - loss: 4.9763e-04
Epoch 9/50
232/232 [==============================] - 0s 2ms/step - loss: 4.4608e-04
Epoch 10/50
232/232 [==============================] - 0s 2ms/step - loss: 4.4402e-04
Epoch 11/50
232/232 [==============================] - 1s 2ms/step - loss: 3.9594e-04
Epoch 12/50
232/232 [==============================] - 0s 2ms/step - loss: 4.11

In [13]:
folder_path = 'model_1'

def save_model(model, folder_path):
    # Create the folder if it doesn't already exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save the model to the folder
    model.save(os.path.join(folder_path, 'model_1.h5'))


save_model(model, folder_path)

In [16]:
#Create an empty list to store predicted values for each player
preds_list = []

for player_id in df['player_name'].unique():
    player_data = df[df['player_name'] == player_id].tail(10)  # select last 10 games for each player
    X = scaler.transform(player_data[["goals", "shots", "xG", "time", "h_goals", "a_goals", "xA", "key_passes", "npg", "npxG", "xGChain", "xGBuildup"]].values)
    
    y = player_data['goals']
    pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2 = r2_score(y, pred)
    # Append the predicted value to the preds_list
    preds_list.append({'player_name': player_id, 'predicted_goals': round(pred[0][0], 2)})

top_predicted = pd.DataFrame(preds_list).sort_values(by='predicted_goals', ascending=False)
print(top_predicted.head(10))

1/1 [==============================] - 0s 32ms/step


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 76ms/step


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 38ms/step


...

1/1 [==============================] - 0s 39ms/step


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 35ms/step


C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


1/1 [==============================] - 0s 35ms/step
              player_name  predicted_goals
265       Phil Foden 6055             3.00
459  Julián Álvarez 10846             1.99
392        Ivan Toney 998             1.98
17       Riyad Mahrez 750             1.96
334    Taiwo Awoniyi 7814             1.92
319   Anthony Gordon 8150             1.02
279    Matheus Cunha 7080             1.01
99       Ryan Fraser 1683             1.01
243       Bobby Reid 6827             1.01
221     Douglas Luiz 6122             1.00


In [17]:
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)

print("Mean squared error: ", mse)
print("Mean absolute error: ", mae)
print("R-squared value: ", r2)

Mean squared error:  8.84146645628641e-05
Mean absolute error:  0.005081435695157807
R-squared value:  0.9992217556365411


Model Training- Train the model selected using different type of parameters which can improve it's performance.

Model Evaluation- Evaluate the performance of the model by testing.  